# Project 1: NYC Air Quality Index Analysis
### Welcome to my first project!

Where do you live in the New York Metropolitan Area — the Upper West Side, Flushing, or maybe Rockaway Beach? Believe it or not, the air you breathe can be quite different depending on where you stand.

In this project, I explore New York City’s Air Quality data from NYC Open Data focusing on Nitrogen Dioxide (NO₂) — one of the key pollutants that shape the city’s air health. Using Python, I clean and visualize the dataset to uncover how NO₂ levels change across neighborhoods, through seasons, and over the years.

You’ll see how places like Flushing and the Upper West Side compare, whether winter air is really worse than summer’s, and how NYC’s air quality has evolved over the past decade.

So grab a cup of coffee ☕, take a deep breath (hopefully a clean one 🌤️), and let’s explore how data can make invisible air visible!

In [109]:
import pandas as pd

Here’s where we get picky with our data. First, we import the dataset and tell Python which number column we actually care about. Then we do a little data “housecleaning” — we only keep the rows that talk about Nitrogen dioxide (NO₂) in 2023, and only for the UHF42 areas (a fancy way of saying the 42 neighborhoods that make up NYC). Basically, we’re trimming out all the extra stuff so we can zoom in on the data that really matters.

In [101]:
# To import the dataset, specify the filename and the target numerical column
df = pd.read_csv('Air_Quality.csv')
target_column = 'Data Value'

# To filter the dataset to get the rows 
# where 'Name' is 'Nitrogen dioxide (NO2)' 
# and 'Time Period' is 'Annual Average 2023'
# and 'Geo Type' is 'UHF42' which divides NYC into 42 distinct neighborhoods
filtered_df = df[(df['Name'] == 'Nitrogen dioxide (NO2)') & 
                 (df['Time Period'] == 'Annual Average 2023') &
                 (df['Geo Type Name'] == 'UHF42')]

## Computing Mean, Median, and Mode
### Pandas Approach
Now that we’ve got our cleaned-up dataset, it’s time to get a quick “data vibe check.” Using pandas, we calculate the mean, median, and mode of our target column to see what the typical air quality values look like. These three stats give us a snapshot of the overall trend — the average level, the middle point, and the most common value. It’s a simple but powerful way to get a feel for the data before diving into visualizations.

In [110]:
# Calculating mean, median, and mode using pandas
mean_value = filtered_df[target_column].mean()
median_value = filtered_df[target_column].median()
mode_pandas = filtered_df[target_column].mode()[0]

print(f"Mean: {mean_value:.2f}")
print(f"Median: {median_value:.2f}")
print(f"Mode (Pandas): {mode_pandas:.2f}")

Mean: 16.79
Median: 16.37
Mode (Pandas): 18.41


### Hard Way Approach
Here, we’re getting hands-on and calculating the mean, median, and mode from scratch — no shortcuts with pandas this time!

First, we pull the numbers we need from our filtered dataset and store them in a list (dropping any missing values along the way). Then:
Mean: We add everything up and divide by the number of data points — the good old average.
Median: We sort the list and find the middle value. If there’s an even number of values, we take the average of the two middle ones.
Mode: We count how often each value appears and pick the one(s) that show up the most. If every value is unique, we note that there’s no mode.

In [119]:
# Extract the data from filtered_df into a list
data = filtered_df[target_column].dropna().tolist()

# Calculate Mean (average)
def calculate_mean(data):
    # Sum all values and divide by count
    total = sum(data)
    count = len(data)
    return total / count

mean_value = calculate_mean(data)
print(f"Mean: {mean_value:.2f}")


# Calculate Median (middle value)
def calculate_median(data):
    # Sort the data first
    sorted_data = sorted(data)
    n = len(sorted_data)
    
    # If odd number of elements, return middle one
    if n % 2 == 1:
        return sorted_data[n // 2]
    # If even number of elements, return average of two middle ones
    else:
        mid1 = sorted_data[n // 2 - 1]
        mid2 = sorted_data[n // 2]
        return (mid1 + mid2) / 2

median_value = calculate_median(data)
print(f"Median: {median_value:.2f}")


# Calculate Mode (most frequent value)
def calculate_mode(data):
    # Count frequency of each value
    frequency = {}
    for value in data:
        if value in frequency:
            frequency[value] += 1
        else:
            frequency[value] = 1
    
    # Find maximum frequency
    if not frequency:
        return None
    
    max_freq = max(frequency.values())
    
    # Find all values with maximum frequency
    modes = [key for key, freq in frequency.items() if freq == max_freq]
    
    # If all values appear only once, there's no mode
    if max_freq == 1:
        return "No mode (all values appear only once)"
    
    return modes

mode_value = calculate_mode(data)
print(f"Mode: {mode_value}")

Mean: 16.79
Median: 16.37
Mode: [18.41]


In [123]:
# Get the filtered data with place names
place_data = filtered_df[['Geo Place Name', target_column]].dropna()
place_data = place_data.sort_values(by=target_column, ascending=False)

print("\n" + "="*70)
print("NYC NEIGHBORHOODS - Nitrogen Dioxide (NO2) Levels")
print("Annual Average 2023")
print("="*70)

# 1. Horizontal Bar Chart by Neighborhood
print("\n[1] BAR CHART - NO2 Levels by Neighborhood")
print("-" * 70)

max_val = place_data[target_column].max()
scale = 40 / max_val  # Scale bars to max 40 characters

for idx, row in place_data.iterrows():
    place_name = row['Geo Place Name']
    value = row[target_column]
    bar_length = int(value * scale)
    bar = '█' * bar_length
    
    # Truncate long names
    display_name = place_name[:25].ljust(25)
    print(f"{display_name} | {bar} {value:.2f}")



NYC NEIGHBORHOODS - Nitrogen Dioxide (NO2) Levels
Annual Average 2023

[1] BAR CHART - NO2 Levels by Neighborhood
----------------------------------------------------------------------
Chelsea - Clinton         | ████████████████████████████████████████ 24.13
Gramercy Park - Murray Hi | ███████████████████████████████████████ 23.60
Greenwich Village - SoHo  | ████████████████████████████████████ 22.07
Lower Manhattan           | ███████████████████████████████████ 21.61
Greenpoint                | ████████████████████████████████ 19.32
Sunset Park               | ███████████████████████████████ 19.11
Upper East Side           | ███████████████████████████████ 18.98
Union Square - Lower East | ███████████████████████████████ 18.97
Long Island City - Astori | ██████████████████████████████ 18.41
West Queens               | ██████████████████████████████ 18.41
Upper West Side           | ██████████████████████████████ 18.38
Downtown - Heights - Slop | ██████████████████████████████ 18.29

In [120]:
# 2. Grouped Visualization (High, Medium, Low)
print("\n[2] GROUPED BY POLLUTION LEVEL")
print("-" * 70)

# Calculate thresholds
q33 = place_data[target_column].quantile(0.33)
q67 = place_data[target_column].quantile(0.67)

high = place_data[place_data[target_column] >= q67]
medium = place_data[(place_data[target_column] >= q33) & 
                    (place_data[target_column] < q67)]
low = place_data[place_data[target_column] < q33]

print(f"\n🔴 HIGH (>= {q67:.2f}):")
for idx, row in high.iterrows():
    print(f"   • {row['Geo Place Name']}: {row[target_column]:.2f}")

print(f"\n🟡 MEDIUM ({q33:.2f} - {q67:.2f}):")
for idx, row in medium.iterrows():
    print(f"   • {row['Geo Place Name']}: {row[target_column]:.2f}")

print(f"\n🟢 LOW (< {q33:.2f}):")
for idx, row in low.iterrows():
    print(f"   • {row['Geo Place Name']}: {row[target_column]:.2f}")


[2] GROUPED BY POLLUTION LEVEL
----------------------------------------------------------------------

🔴 HIGH (>= 17.43):
   • Chelsea - Clinton: 24.13
   • Gramercy Park - Murray Hill: 23.60
   • Greenwich Village - SoHo: 22.07
   • Lower Manhattan: 21.61
   • Greenpoint: 19.32
   • Sunset Park: 19.11
   • Upper East Side: 18.98
   • Union Square - Lower East Side: 18.97
   • Long Island City - Astoria: 18.41
   • West Queens: 18.41
   • Upper West Side: 18.38
   • Downtown - Heights - Slope: 18.29
   • Williamsburg - Bushwick: 18.16
   • Hunts Point - Mott Haven: 17.53

🟡 MEDIUM (15.50 - 17.43):
   • East New York: 17.34
   • Crotona -Tremont: 17.27
   • High Bridge - Morrisania: 17.08
   • Washington Heights: 17.05
   • Bedford Stuyvesant - Crown Heights: 16.99
   • Ridgewood - Forest Hills: 16.73
   • Bensonhurst - Bay Ridge: 16.39
   • Borough Park: 16.35
   • East Harlem: 16.34
   • Central Harlem - Morningside Heights: 16.29
   • Port Richmond: 16.28
   • East Flatbush - Flatbu

In [122]:
# 5. Comparative Scale
print("\n[5] COMPARATIVE SCALE (Relative to Mean)")
print("-" * 70)

mean_val = place_data[target_column].mean()
print(f"Mean NO2 Level: {mean_val:.2f}\n")

# Show neighborhoods above and below mean
above_mean = place_data[place_data[target_column] >= mean_val]
below_mean = place_data[place_data[target_column] < mean_val]

print(f"Above Mean ({len(above_mean)} neighborhoods):")
for idx, row in above_mean.head(15).iterrows():
    diff = row[target_column] - mean_val
    plus_signs = '+' * int(diff * 2)
    print(f"  {row['Geo Place Name'][:28]:28s} {plus_signs:>15s} +{diff:5.2f}")

print(f"\nBelow Mean ({len(below_mean)} neighborhoods):")
for idx, row in below_mean.tail(15).iterrows():
    diff = mean_val - row[target_column]
    minus_signs = '-' * int(diff * 2)
    print(f"  {row['Geo Place Name'][:28]:28s} {minus_signs:>15s} -{diff:5.2f}")

# Summary statistics
print("\n" + "="*70)
print(f"Total Neighborhoods Analyzed: {len(place_data)}")
print(f"Highest: {place_data.iloc[0]['Geo Place Name']} ({place_data.iloc[0][target_column]:.2f})")
print(f"Lowest: {place_data.iloc[-1]['Geo Place Name']} ({place_data.iloc[-1][target_column]:.2f})")
print(f"Mean: {mean_val:.2f}")
print(f"Median: {place_data[target_column].median():.2f}")
print("="*70)


[5] COMPARATIVE SCALE (Relative to Mean)
----------------------------------------------------------------------
Mean NO2 Level: 16.79

Above Mean (19 neighborhoods):
  Chelsea - Clinton             ++++++++++++++ + 7.34
  Gramercy Park - Murray Hill    +++++++++++++ + 6.81
  Greenwich Village - SoHo          ++++++++++ + 5.28
  Lower Manhattan                    +++++++++ + 4.82
  Greenpoint                             +++++ + 2.53
  Sunset Park                             ++++ + 2.32
  Upper East Side                         ++++ + 2.19
  Union Square - Lower East Si            ++++ + 2.18
  Long Island City - Astoria               +++ + 1.62
  West Queens                              +++ + 1.62
  Upper West Side                          +++ + 1.59
  Downtown - Heights - Slope               +++ + 1.50
  Williamsburg - Bushwick                   ++ + 1.37
  Hunts Point - Mott Haven                   + + 0.74
  East New York                              + + 0.55

Below Mean (23 neighbo